In [7]:
from random import shuffle
import glob, h5py, pickle, cv2, re
import numpy as np
from IPython.display import clear_output
from pandas import read_csv
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from collections import Counter
width = 128
height = 128
channels = 3
sub = 'projected_numelem'

trainpath = '/mnt/HDD04/Projection_data/Scripts/Python/datasets/Angle_Projection_New/Angle_Projection_New/'
filename = 'datasets/' + sub + '.hdf5'
print('File to be created: ' + filename)

File to be created: datasets/projected_numelem.hdf5


In [8]:
num_elem = [4, 16, 32, 72, 86]
# modes = ['train', 'test']

for n in range(len(num_elem)):
    filename = 'datasets/' + sub + '.hdf5'
    filename = filename.replace('.hdf5', str(num_elem[n]) + '.hdf5')
    
    train_files = glob.glob(trainpath + 'train*' + str(num_elem[n]) + '/*.png')
#     print('Num files for ' + str(modes[m]) + ', num_elem: ' + str(num_elem[n]) + ', ' + str(len(files)))
    train_labels = []
    train_images = []
    cnt = 1
    
    for i in range(len(train_files)):
        fname = train_files[i].split('/')[-1]
        if '\\' in fname:
            fname = fname.split('\\')[-1]
#         print(fname)
        if '_orig' in fname:
            continue
        class_idx = [m.end() for m in re.finditer('class', fname)]
        ang_idx = [m.end() for m in re.finditer('ang', fname)]
        underscore_idx = [m.start() for m in re.finditer('_', fname)]
#             print('class_idx', class_idx)
#             print('underscore_idx', underscore_idx)

        classes = []
        classes.append(int(fname[class_idx[0]: underscore_idx[1]]))
        classes.append(int(fname[class_idx[1]: underscore_idx[5]]))
        if len(underscore_idx) == 12:
            classes.append(int(fname[class_idx[2]: underscore_idx[9]]))
#             print('classes', classes)
        angs = []
        angs.append(int(fname[ang_idx[0]: underscore_idx[2]]))
        angs.append(int(fname[ang_idx[1]: underscore_idx[6]]))
        if len(underscore_idx) == 12:
            angs.append(int(fname[ang_idx[2]: underscore_idx[10]]))
#             print('angs', angs)
        proj_ang = int(fname[fname.find('proj')+4: -4])
#             print('proj_ang', proj_ang)
        if proj_ang == angs[0]:
            clas = classes[0]
        elif proj_ang == angs[1]:
            clas = classes[1]
        elif proj_ang == angs[2]:
            clas = classes[2]

        if clas == 13:
            clas = 8
        elif clas == 14:
            clas = 9
        train_labels.append(clas-1)
#             print('clas-1', clas-1)
        img = cv2.imread(train_files[i])
        img = cv2.resize(img, (width, height), interpolation=cv2.INTER_CUBIC)# resize to (128,128)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 load images as BGR, convert it to RGB
        clear_output(wait=True)
        print('Loading image '+str(cnt)+'/'+str(len(train_files))+ ' ' + fname)
        train_images.append(img)  
        cnt += 1
    
    print(np.unique(train_labels))
    train_labels = to_categorical(np.array(train_labels))
    train_images = np.swapaxes(np.array(train_images), 1, 2).reshape(len(train_images), width, height, channels)/255.
    print(train_images.shape)
    print(train_labels.shape)

    test_files = glob.glob(trainpath + 'test*' + str(num_elem[n]) + '/*.png')
#     print('Num files for ' + str(modes[m]) + ', num_elem: ' + str(num_elem[n]) + ', ' + str(len(files)))
    test_labels = []
    test_images = []
    cnt = 1
    
    for i in range(len(test_files)):
        fname = test_files[i].split('/')[-1]
        if '\\' in fname:
            fname = fname.split('\\')[-1]
#         print(fname)
        if '_orig' in fname:
            continue
        class_idx = [m.end() for m in re.finditer('class', fname)]
        ang_idx = [m.end() for m in re.finditer('ang', fname)]
        underscore_idx = [m.start() for m in re.finditer('_', fname)]
#             print('class_idx', class_idx)
#             print('underscore_idx', underscore_idx)

        classes = []
        classes.append(int(fname[class_idx[0]: underscore_idx[1]]))
        classes.append(int(fname[class_idx[1]: underscore_idx[5]]))
        if len(underscore_idx) == 12:
            classes.append(int(fname[class_idx[2]: underscore_idx[9]]))
#             print('classes', classes)
        angs = []
        angs.append(int(fname[ang_idx[0]: underscore_idx[2]]))
        angs.append(int(fname[ang_idx[1]: underscore_idx[6]]))
        if len(underscore_idx) == 12:
            angs.append(int(fname[ang_idx[2]: underscore_idx[10]]))
#             print('angs', angs)
        proj_ang = int(fname[fname.find('proj')+4: -4])
#             print('proj_ang', proj_ang)
        if proj_ang == angs[0]:
            clas = classes[0]
        elif proj_ang == angs[1]:
            clas = classes[1]
        elif proj_ang == angs[2]:
            clas = classes[2]

        if clas == 13:
            clas = 8
        elif clas == 14:
            clas = 9
        test_labels.append(clas-1)
#             print('clas-1', clas-1)
        img = cv2.imread(test_files[i])
        img = cv2.resize(img, (width, height), interpolation=cv2.INTER_CUBIC)# resize to (128,128)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # cv2 load images as BGR, convert it to RGB
        clear_output(wait=True)
        print('Loading image '+str(cnt)+'/'+str(len(test_files))+ ' ' + fname)
        test_images.append(img)  
        cnt += 1
    
    print(np.unique(test_labels))
    test_labels = to_categorical(np.array(test_labels))
    test_images = np.swapaxes(np.array(test_images), 1, 2).reshape(len(test_images), width, height, channels)/255.
    print(test_images.shape)
    print(test_labels.shape)

    f = h5py.File(filename, mode='w')
    f.create_dataset("train_img", train_images.shape, np.uint8)
    f.create_dataset("test_img", test_images.shape, np.uint8)  
    f.create_dataset("train_labels", train_labels.shape, np.uint8)
    f.create_dataset("test_labels", test_labels.shape, np.uint8)

    f["train_img"][...] = train_images
    f["test_img"][...] = test_images
    f["train_labels"][...] = train_labels
    f["test_labels"][...] = test_labels
    f.close()
    print(filename+' created.')

Loading image 350/500 pCemre_class1_ang0_iter5_pSpot_class3_ang-45_iter1_proj0.png
[0 1 2 3 4 5 6 7 8]
(350, 128, 128, 3)
(350, 9)
datasets/projected_numelem86.hdf5 created.
